## Testing the main scripts

In [49]:
import datetime
import os
import sys
import torch

from torch.utils.data import DataLoader
from torchvision import transforms

sys.path.append('d:/gsn-projekt')

import amphibian.preprocess.preprocessing as preproc

from amphibian.fetch.reader import AmphibianReader
from amphibian.preprocess.train_test_split import TrainTestSplit

In [15]:
ar = AmphibianReader('../data/all_values/banking', datetime.datetime(2010,1,4), datetime.datetime(2018, 12, 31))

In [16]:
_ = ar.create_torch()

In [12]:
ar.torch['AMERICA'].shape

torch.Size([2342, 6, 10])

In [ ]:
ar.torch['AMERICA'][2000:2342, :, :]

In [51]:
ar.torch['ASIA_PACIFIC'].shape

torch.Size([2342, 6, 9])

## Balancing classes

In [19]:
tts = TrainTestSplit(ar, int_start=0, int_end=2342, input_reg='ASIA_PACIFIC', pred_reg='EMEIA')

In [21]:
eps = 0.01

In [26]:
tsds = preproc.TimeSeriesDataset(
    tts, int_len=10, 
    transform=transforms.Compose([
        preproc.FillNaN(), preproc.Normalizing(),
        preproc.DummyFillNaN(), preproc.Formatting(),
        preproc.FormattingY()
    ])
)

d:\gsn-projekt\venv\lib\site-packages\torch\tensor.py:339: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [27]:
len(tsds)

18640

In [44]:
dl = DataLoader(tsds, batch_size=len(tsds))
for _, batch in enumerate(dl):
    print(torch.sum(batch['train_y'] == 0))
    print(torch.sum(batch['train_y'] == 1))
    print(torch.sum(batch['train_y'] == 2))

tensor(6959, device='cuda:0')
tensor(4587, device='cuda:0')
tensor(7094, device='cuda:0')


The default threshold of 50bp can remain as is.

In [47]:
run_path = r'data/run_attention_{:%Y%m%d_%H%M%S}.csv'.format(datetime.datetime.now())

In [48]:
run_path

'data/run_attention_20190611_003855.csv'

In [50]:
os.makedirs(run_path)